# LightGBM model training

This notebook connects to the project Postgres database, pulls transactions, performs per-user feature engineering, trains a LightGBM model to predict a credit score (or proxy), and saves the model to `ml/model.lgb`.

Notes:
- If you have a ground-truth `score` column for each user in your DB, use that as the target.
- Otherwise, this notebook demonstrates creating a proxy target for demonstration only. Replace with a real label for production training.

In [ ]:
# Install dependencies in your environment before running: polars, psycopg2-binary, lightgbm, scikit-learn
import os
import polars as pl
import psycopg2
import lightgbm as lgb
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# DB connection - ensure DATABASE_URL is set in your environment
db_url = os.environ.get('DATABASE_URL')
assert db_url, 'Set DATABASE_URL before running this notebook'
conn = psycopg2.connect(db_url)
cur = conn.cursor()
# Pull transactions - adjust column names if your schema differs
cur.execute('SELECT 
, date, description, amount, type FROM 
 ORDER BY "userId", date ASC')
rows = cur.fetchall()
conn.close()
df = pl.DataFrame(rows, schema=['userId','date','description','amount','type'])
print('Total rows', df.height)

In [ ]:
# Feature engineering per user
def compute_user_features(user_df: pl.DataFrame) -> dict:
    # Very similar to the server feature logic in api/python/get_score.py
    if user_df.height == 0:
        return { 'avg_monthly_income':0, 'avg_monthly_expense':0, 'savings_rate':0, 'expense_to_income_ratio':0, 'num_loan_payments':0, 'pct_spend_on_food':0, 'total_transactions':0 }
    if 'date' in user_df.columns:
        user_df = user_df.with_column(pl.col('date').str.strptime(pl.Datetime, fmt=None).alias('date'))
        user_df = user_df.with_column(pl.col('date').dt.strftime('%Y-%m').alias('month'))
    else:
        user_df = user_df.with_column(pl.lit('unknown').alias('month'))
    # cast amount
    user_df = user_df.with_column(pl.col('amount').cast(pl.Float64))
    monthly = user_df.groupby(['month','type']).agg(pl.col('amount').sum().alias('sum_amount'))
    try:
        income = monthly.filter(pl.col('type') == 'Credit').select('sum_amount').to_series().to_list()
        expense = monthly.filter(pl.col('type') == 'Debit').select('sum_amount').to_series().to_list()
    except Exception:
        income = []
        expense = []
    avg_income = float(sum(income)/len(income)) if income else 0
    avg_expense = float(sum(expense)/len(expense)) if expense else 0
    savings_rate = ((avg_income - avg_expense)/avg_income*100) if avg_income else 0
    expense_to_income_ratio = (avg_expense/avg_income) if avg_income else 0
    descs = [ (d or '').lower() for d in user_df.get_column('description').to_list() ]
    loan_count = sum(1 for d in descs if any(k in d for k in ['emi','loan','instal']))
    total_debit = sum(abs(x) for x,t in zip(user_df.get_column('amount').to_list(), user_df.get_column('type').to_list()) if t=='Debit')
    food_spend = sum(abs(x) for x,t,d in zip(user_df.get_column('amount').to_list(), user_df.get_column('type').to_list(), descs) if t=='Debit' and any(k in d for k in ['zomato','swiggy','food','restaurant']))
    pct_food = (food_spend/total_debit*100) if total_debit else 0
    return {'avg_monthly_income':avg_income, 'avg_monthly_expense':avg_expense, 'savings_rate':savings_rate, 'expense_to_income_ratio':expense_to_income_ratio, 'num_loan_payments':loan_count, 'pct_spend_on_food':pct_food, 'total_transactions':user_df.height}

# Build features for all users
users = df.select('userId').unique().to_series().to_list()
rows = []
for u in users:
    udf = df.filter(pl.col('userId') == u)
    feats = compute_user_features(udf)
    feats['userId']=u
    rows.append(feats)
fdf = pl.DataFrame(rows)
print('Users:', fdf.height)

In [ ]:
# Prepare training data - replace `target` with real labels if available. Here we create a proxy target for demonstration.
# Proxy: higher savings_rate and lower expense_to_income_ratio -> higher score
import numpy as np
fdf = fdf.with_column((50 + (fdf['savings_rate']*2) - (fdf['expense_to_income_ratio']*20)).alias('target_proxy'))
X = fdf.select(['avg_monthly_income','avg_monthly_expense','savings_rate','expense_to_income_ratio','num_loan_payments','pct_spend_on_food','total_transactions']).to_pandas()
y = fdf.select('target_proxy').to_series().to_list()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test)
params = { 'objective':'regression', 'metric':'rmse', 'verbosity':-1 }
bst = lgb.train(params, train_data, valid_sets=[train_data, valid_data], num_boost_round=100, early_stopping_rounds=10)
preds = bst.predict(X_test)
print('RMSE', mean_squared_error(y_test, preds, squared=False))
# Save model to ml/model.lgb
os.makedirs('ml', exist_ok=True)
bst.save_model('ml/model.lgb')
print('Saved model to ml/model.lgb')